In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from torch.nn.functional import softmax
from torch.optim import Adam

In [7]:
# u.data：user id | item id | rating | timestamp
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user id','item id','rating','timestamp'], encoding = "UTF-8")

'''u.item     -- Information about the items (movies); this is a tab separated
              list of
              movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |
              The last 19 fields are the genres, a 1 indicates the movie
              is of that genre, a 0 indicates it is not; movies can be in
              several genres at once.
              The movie ids are the ones used in the u.data data set.
'''
movies = pd.read_csv('ml-100k/u.item', sep='|', names=["movie id","movie title","release date","video release date","IMDb URL",
                                                        "unknown","Action","Adventure","Animation","Children's","Comedy","Crime",
                                                        " Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery",
                                                        "Romance","Sci-Fi","Thriller","War","Western" ], encoding = "ISO-8859-1")
'''u.user     -- Demographic information about the users; this is a tab
              separated list of
              user id | age | gender | occupation | zip code
              The user ids are the ones used in the u.data data set.
'''
users = pd.read_csv('ml-100k/u.user', sep='|', names=['user id','age','gender','occupation','zip code'], encoding = "ISO-8859-1")
print("=====================ratings==========================")
print(ratings.head())
print("=====================movies==========================")
print(movies.head())
print("=====================users==========================")
print(users.head())

=====================ratings==========================
   user id  item id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596
=====================movies==========================
   movie id        movie title release date  video release date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact

In [8]:
Users = users.drop("zip code",axis = 1)
# 将性别列转换为 0/1 变量
Users['gender'] = Users['gender'].map({'M': 1, 'F': 0})
# 将职业列按顺序进行数字编码
occupations = Users['occupation'].unique()
occupation_mapping = {occ: i+1 for i, occ in enumerate(occupations)}
Users['occupation'] = Users['occupation'].map(occupation_mapping)

# 将Users写入CSV文件
Users.to_csv('Users.csv', index=True)
print(Users.head())

   user id  age  gender  occupation
0        1   24       1           1
1        2   53       0           2
2        3   23       1           3
3        4   24       1           1
4        5   33       0           2


In [9]:
Ratings = ratings.drop("timestamp",axis = 1)
Ratings = Ratings.rename(columns={'item id': 'movie id'})
print(Ratings.head())

   user id  movie id  rating
0      196       242       3
1      186       302       3
2       22       377       1
3      244        51       2
4      166       346       1


In [10]:
# 移除unknown是因为unknown可以用其他类别特征都为0表示
Movies = movies.drop(["video release date","IMDb URL","release date","movie title","unknown"],axis = 1)
Movies.to_csv('Movies.csv', index=True)
print(Movies.head())

   movie id  Action  Adventure  Animation  Children's  Comedy  Crime  \
0         1       0          0          1           1       1      0   
1         2       1          1          0           0       0      0   
2         3       0          0          0           0       0      0   
3         4       1          0          0           0       1      0   
4         5       0          0          0           0       0      1   

    Documentary  Drama  Fantasy  Film-Noir  Horror  Musical  Mystery  Romance  \
0             0      0        0          0       0        0        0        0   
1             0      0        0          0       0        0        0        0   
2             0      0        0          0       0        0        0        0   
3             0      1        0          0       0        0        0        0   
4             0      1        0          0       0        0        0        0   

   Sci-Fi  Thriller  War  Western  
0       0         0    0        0  
1       

In [11]:
merged_data = pd.merge(Ratings, Users, on='user id')
data = pd.merge(merged_data, Movies, on='movie id')
print(data.head())

   user id  movie id  rating  age  gender  occupation  Action  Adventure  \
0      196       242       3   49       1           3       0          0   
1      186       302       3   39       0           4       0          0   
2       22       377       1   25       1           3       0          0   
3      244        51       2   28       1           1       0          0   
4      166       346       1   47       1           8       0          0   

   Animation  Children's  ...  Fantasy  Film-Noir  Horror  Musical  Mystery  \
0          0           0  ...        0          0       0        0        0   
1          0           0  ...        0          1       0        0        1   
2          0           1  ...        0          0       0        0        0   
3          0           0  ...        0          0       0        0        0   
4          0           0  ...        0          0       0        0        0   

   Romance  Sci-Fi  Thriller  War  Western  
0        0       0     

In [13]:
# 找出UserID列和MovieID列的最大值并打印
max_user_id = data['user id'].max()
max_movie_id = data['movie id'].max()
print(f'Max UserID: {max_user_id}')
print(f'Max MovieID: {max_movie_id}')
# 将数据分为特征和标签
y = data['rating']
X = data.drop('rating',axis = 1)
# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_tensor = torch.tensor(X_train.values).int()
X_test_tensor = torch.tensor(X_test.values).int()
y_train_tensor = torch.tensor(y_train.values).float()
y_test_tensor = torch.tensor(y_test.values).float()
print(X_train_tensor.dtype)
print(y_test_tensor.dtype)

Max UserID: 943
Max MovieID: 1682
torch.int32
torch.float32


In [14]:
header = list(X.columns)
print(header)
print(X_train_tensor.size())

['user id', 'movie id', 'age', 'gender', 'occupation', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', ' Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
torch.Size([80000, 23])


In [15]:
class RatingsDataset(Dataset):
    def __init__(self, features, labels):
        """
        Args:
            features (Tensor): 特征数据张量。
            labels (Tensor): 标签数据张量。
        """
        self.features = features
        self.labels = labels

    def __len__(self):
        """返回数据集中的数据点总数"""
        return len(self.labels)

    def __getitem__(self, idx):
        """根据给定的索引 idx 返回特征和标签"""
        return self.features[idx], self.labels[idx]

In [18]:
# 创建训练集和测试集的 Dataset
train_dataset = RatingsDataset(X_train_tensor, y_train_tensor)
test_dataset = RatingsDataset(X_test_tensor, y_test_tensor)

# 创建DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=8, shuffle=False)


In [19]:
import torch
import torch.nn as nn

class RatingPredictor(nn.Module):
    def __init__(self, num_users, num_movies, user_embedding_dim, movie_embedding_dim, user_feature_dim, movie_feature_dim, nhead,num_encoder_layers,dim_feedforward, dropout_rate):
        super(RatingPredictor, self).__init__()

        self.user_embedding = nn.Embedding(num_embeddings=num_users + 1, embedding_dim=user_embedding_dim)
        self.movie_embedding = nn.Embedding(num_embeddings=num_movies + 1, embedding_dim=movie_embedding_dim)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=user_embedding_dim + movie_embedding_dim + user_feature_dim + movie_feature_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout_rate
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers= num_encoder_layers)

        self.fc = nn.Linear(user_embedding_dim + movie_embedding_dim + user_feature_dim + movie_feature_dim, 1)

    def forward(self, user_ids, movie_ids, user_features, movie_features):
        user_embed = self.user_embedding(user_ids)
        movie_embed = self.movie_embedding(movie_ids)
        
        # 用户ID的Embedding与用户特征拼接
        user_combined_features = torch.cat([user_embed, user_features], dim=-1)
        # 电影ID的Embedding与电影特征拼接
        movie_combined_features = torch.cat([movie_embed, movie_features], dim=-1)
        
        # 将用户和电影的组合特征拼接
        combined_features = torch.cat([user_combined_features, movie_combined_features], dim=-1).unsqueeze(1)  # 增加一个维度以符合Transformer输入要求
        
        # 通过Transformer Encoder
        transformer_output = self.transformer_encoder(combined_features)
        transformer_output = transformer_output.squeeze(1)  # 移除多余的维度
        
        # 通过全连接层
        output = self.fc(transformer_output)

        return output.squeeze()

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [21]:
# 模型超参数
# num_users, num_movies, user_embedding_dim=32, 
# movie_embedding_dim=32, user_feature_dim=3, movie_feature_dim=18, nhead=4, dim_feedforward=2048, dropout_rate=0.1
num_users = 943
num_movies = 1862
num_encoder_layers = 1
user_embedding_dim=32
movie_embedding_dim=32
user_feature_dim=3
movie_feature_dim=18
# dmodel = 85
nhead=5
dim_feedforward=1024#太大就会用太多计算资源
dropout_rate=0.1# 根据你的数据集进行调整

model = RatingPredictor(
    num_users=num_users,
    num_movies=num_movies,
    user_embedding_dim=user_embedding_dim,
    movie_embedding_dim=movie_embedding_dim,
    user_feature_dim=user_feature_dim,
    movie_feature_dim=movie_feature_dim,
    nhead=nhead,
    num_encoder_layers=num_encoder_layers,
    dim_feedforward=dim_feedforward,
    dropout_rate=dropout_rate
)

print(model)


RatingPredictor(
  (user_embedding): Embedding(944, 32)
  (movie_embedding): Embedding(1863, 32)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=85, out_features=85, bias=True)
        )
        (linear1): Linear(in_features=85, out_features=1024, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=1024, out_features=85, bias=True)
        (norm1): LayerNorm((85,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((85,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc): Linear(in_features=85, out_features=1, bias=True)
)


C:\Users\pang_\.conda\envs\py10\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [22]:
# 模型评估
def model_eval(model,test_loader):
    model.eval()  # 将模型置于评估模式

    # 在评估之前定义一些变量以记录评估结果
    running_mse = 0.0
    total_samples = 0

    with torch.no_grad():  # 禁用梯度计算
        for features, labels in test_loader:
            user_ids = features[:, 0].long().to(device)  # 用户ID
            movie_ids = features[:, 1].long().to(device)  # 电影ID
            user_feats = features[:, 2:5].to(device)  # 用户特征
            movie_feats = features[:, 5:].to(device)  # 电影特征
            labels = labels.to(device)

            predictions = model(user_ids, movie_ids, user_feats, movie_feats)
            prediction_clipped = predictions.clamp(min=1, max=5)
            
            # 计算均方误差
            loss = criterion(prediction_clipped, labels)
            running_mse += loss.item() * features.size(0)
            total_samples += features.size(0)

    # 计算平均均方误差
    mse = running_mse / total_samples
#     rmse = torch.sqrt(mse)
    return mse

In [30]:
# epoch = 300 其实是301轮
PATH = './checkpoint/rating_regressor_300.pth'
model.load_state_dict(torch.load(PATH))
model.to(device)
criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = Adam(model.parameters(), lr=0.001)
for epoch in range(1):
    model.train()
    running_loss = 0.0
    total_loss = 0
    for i, data in enumerate(train_loader,0):
        features, labels = data
        user_ids = features[:, 0].long().to(device)  # 用户ID
        movie_ids = features[:, 1].long().to(device)  # 电影ID
        user_feats = features[:, 2:5].to(device)  # 用户特征
        movie_feats = features[:, 5:].to(device)  # 电影特征
        print(user_ids,movie_ids,user_feats,movie_feats)
        print(user_ids.size(),movie_ids.size(),user_feats.size(),movie_feats.size())
        break

tensor([418, 815, 897, 606, 118, 640, 116, 642], device='cuda:0') tensor([269, 380,  95, 637, 551, 663, 661, 102], device='cuda:0') tensor([[55,  0, 17],
        [32,  1,  2],
        [30,  1,  2],
        [28,  1, 11],
        [21,  1,  5],
        [20,  1,  6],
        [40,  1, 18],
        [18,  0,  6]], device='cuda:0', dtype=torch.int32) tensor([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       device='cuda:0', dtype=torch.int32)
torch.Size([8]) torch.Size([8]) torch.Size([8, 3]) torch.Size([8, 18])


In [15]:
# 训练

# 初始化 TensorBoard writer
writer = SummaryWriter(log_dir='./log')

model.to(device)
criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = Adam(model.parameters(), lr=0.001)

num_epochs = 100

n_iter = 0
Mse =[]

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_loss = 0
    for i, data in enumerate(train_loader,0):
        features, labels = data
        user_ids = features[:, 0].long().to(device)  # 用户ID
        movie_ids = features[:, 1].long().to(device)  # 电影ID
        user_feats = features[:, 2:5].to(device)  # 用户特征
        movie_feats = features[:, 5:].to(device)  # 电影特征
        labels = labels.to(device)

        optimizer.zero_grad()
        predictions = model(user_ids, movie_ids, user_feats, movie_feats)
        prediction_clipped = predictions.clamp(min=1, max=5)
        loss = criterion(prediction_clipped, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        total_loss += loss.item()
        if i % 200 == 199:
            n_iter += 1
            current_loss = running_loss / 200
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {current_loss:.3f}')
            writer.add_scalar('Loss/train', current_loss, n_iter)
            running_loss = 0.0
        
    if epoch % 10 == 0:
    # 保存模型
        PATH = './checkpoint/rating_regressor_%d.pth' % epoch
        torch.save(model.state_dict(), PATH)
        
    avg_loss = total_loss / len(train_loader)
    mse = model_eval(model,test_loader)
    Mse.append(mse)
    print(f'Epoch {epoch+1}, Average Loss: {avg_loss:.4f},MSE:{mse}')

C:\Users\pang_\.conda\envs\py10\lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch 1, Average Loss: 1.2035,MSE:1.1314000712901353
Epoch 2, Average Loss: 1.0415,MSE:0.9893959114834666
Epoch 3, Average Loss: 0.9789,MSE:0.9135288128580898
Epoch 4, Average Loss: 0.9560,MSE:0.9480244465880096
Epoch 5, Average Loss: 0.9345,MSE:0.9040041882034391
Epoch 6, Average Loss: 0.9202,MSE:0.9248678304813802
Epoch 7, Average Loss: 0.9001,MSE:0.8657799772314727
Epoch 8, Average Loss: 0.8904,MSE:0.8704155466578901
Epoch 9, Average Loss: 0.8767,MSE:0.8364718932341785
Epoch 10, Average Loss: 0.8674,MSE:0.8426222413949669
Epoch 11, Average Loss: 0.8573,MSE:0.8694272248327732
Epoch 12, Average Loss: 0.8535,MSE:0.8402617111448198
Epoch 13, Average Loss: 0.8480,MSE:0.8119284650109708
Epoch 14, Average Loss: 0.8415,MSE:0.8600026292067021
Epoch 15, Average Loss: 0.8389,MSE:0.8142544409230351
Epoch 16, Average Loss: 0.8375,MSE:0.8075948779158294
Epoch 17, Average Loss: 0.8329,MSE:0.8250312206327916
Epoch 18, Average Loss: 0.8274,MSE:0.8033192804407329
Epoch 19, Average Loss: 0.8240,MSE:0.

In [18]:
for epoch in range(100,300):
    model.train()
    running_loss = 0.0
    total_loss = 0
    for i, data in enumerate(train_loader,0):
        features, labels = data
        user_ids = features[:, 0].long().to(device)  # 用户ID
        movie_ids = features[:, 1].long().to(device)  # 电影ID
        user_feats = features[:, 2:5].to(device)  # 用户特征
        movie_feats = features[:, 5:].to(device)  # 电影特征
        labels = labels.to(device)

        optimizer.zero_grad()
        predictions = model(user_ids, movie_ids, user_feats, movie_feats)
        prediction_clipped = predictions.clamp(min=1, max=5)
        loss = criterion(prediction_clipped, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        total_loss += loss.item()
        if i % 200 == 199:
            n_iter += 1
            current_loss = running_loss / 200
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {current_loss:.3f}')
            writer.add_scalar('Loss/train', current_loss, n_iter)
            running_loss = 0.0
        
    if epoch % 10 == 0:
    # 保存模型
        PATH = './checkpoint/rating_regressor_%d.pth' % epoch
        torch.save(model.state_dict(), PATH)
        
    avg_loss = total_loss / len(train_loader)
    mse = model_eval(model,test_loader)
    Mse.append(mse)
    print(f'Epoch {epoch+1}, Average Loss: {avg_loss:.4f},MSE:{mse}')

Epoch 101, Average Loss: 0.6776,MSE:0.6174963885931298
Epoch 102, Average Loss: 0.6702,MSE:0.6156615067165345
Epoch 103, Average Loss: 0.6700,MSE:0.6103940164234489
Epoch 104, Average Loss: 0.6713,MSE:0.6302837017495185
Epoch 105, Average Loss: 0.6720,MSE:0.6225634551256896
Epoch 106, Average Loss: 0.6703,MSE:0.6074131921777501
Epoch 107, Average Loss: 0.6707,MSE:0.6317069182826206
Epoch 108, Average Loss: 0.6649,MSE:0.6238227596476674
Epoch 109, Average Loss: 0.6694,MSE:0.6091211354006082
Epoch 110, Average Loss: 0.6635,MSE:0.6087109590530395
Epoch 111, Average Loss: 0.6668,MSE:0.6039764592236839
Epoch 112, Average Loss: 0.6622,MSE:0.6274332544805482
Epoch 113, Average Loss: 0.6656,MSE:0.6145241071160883
Epoch 114, Average Loss: 0.6611,MSE:0.610173229342699
Epoch 115, Average Loss: 0.6632,MSE:0.6032706308657303
Epoch 116, Average Loss: 0.6596,MSE:0.6000351243756712
Epoch 117, Average Loss: 0.6592,MSE:0.6041502215078101
Epoch 118, Average Loss: 0.6562,MSE:0.6216999321531504
Epoch 119, 

Epoch 251, Average Loss: 0.5850,MSE:0.5170409189381636
Epoch 252, Average Loss: 0.5841,MSE:0.5109746322956867
Epoch 253, Average Loss: 0.5876,MSE:0.5099036626946181
Epoch 254, Average Loss: 0.5863,MSE:0.5154953235295601
Epoch 255, Average Loss: 0.5847,MSE:0.5315893378846347
Epoch 256, Average Loss: 0.5819,MSE:0.5510127036408521
Epoch 257, Average Loss: 0.5803,MSE:0.510504874435626
Epoch 258, Average Loss: 0.5788,MSE:0.5037898506363854
Epoch 259, Average Loss: 0.5884,MSE:0.5188578464061022
Epoch 260, Average Loss: 0.5831,MSE:0.5260272380075417
Epoch 261, Average Loss: 0.5818,MSE:0.5035707966405898
Epoch 262, Average Loss: 0.5857,MSE:0.5216370046235621
Epoch 263, Average Loss: 0.5870,MSE:0.5098203676672652
Epoch 264, Average Loss: 0.5904,MSE:0.520437976229377
Epoch 265, Average Loss: 0.5964,MSE:0.5569791628439911
Epoch 266, Average Loss: 0.5896,MSE:0.5202394808024168
Epoch 267, Average Loss: 0.5902,MSE:0.5181577700207941
Epoch 268, Average Loss: 0.5846,MSE:0.5240717034007423
Epoch 269, A

In [19]:
print(epoch,n_iter)

299 15000


In [20]:
# 初始化 TensorBoard writer
writer = SummaryWriter(log_dir='./log')

# epoch = 300 其实是301轮
PATH = './checkpoint/rating_regressor_300.pth'
model.load_state_dict(torch.load(PATH))
n_iter = 15000
model.to(device)
criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = Adam(model.parameters(), lr=0.001)
Mse = []

In [21]:
for epoch in range(300,500):
    model.train()
    running_loss = 0.0
    total_loss = 0
    for i, data in enumerate(train_loader,0):
        features, labels = data
        user_ids = features[:, 0].long().to(device)  # 用户ID
        movie_ids = features[:, 1].long().to(device)  # 电影ID
        user_feats = features[:, 2:5].to(device)  # 用户特征
        movie_feats = features[:, 5:].to(device)  # 电影特征
        labels = labels.to(device)

        optimizer.zero_grad()
        predictions = model(user_ids, movie_ids, user_feats, movie_feats)
        prediction_clipped = predictions.clamp(min=1, max=5)
        loss = criterion(prediction_clipped, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        total_loss += loss.item()
        if i % 200 == 199:
            n_iter += 1
            current_loss = running_loss / 200
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {current_loss:.3f}')
            writer.add_scalar('Loss/train', current_loss, n_iter)
            running_loss = 0.0
        
    if epoch % 10 == 0 or epoch == 499:
    # 保存模型
        PATH = './checkpoint/rating_regressor_%d.pth' % epoch
        torch.save(model.state_dict(), PATH)
        
    avg_loss = total_loss / len(train_loader)
    mse = model_eval(model,test_loader)
    Mse.append(mse)
    print(f'Epoch {epoch+1}, Average Loss: {avg_loss:.4f},MSE:{mse}')


Epoch 301, Average Loss: 0.5721,MSE:0.5102098921283148
Epoch 302, Average Loss: 0.5720,MSE:0.49653612046614287
Epoch 303, Average Loss: 0.5689,MSE:0.49656740480400624
Epoch 304, Average Loss: 0.5662,MSE:0.5004581382468343
Epoch 305, Average Loss: 0.5654,MSE:0.4891149421745911
Epoch 306, Average Loss: 0.5658,MSE:0.4862265779586509
Epoch 307, Average Loss: 0.5692,MSE:0.49655870473962277
Epoch 308, Average Loss: 0.5668,MSE:0.4795576021321118
Epoch 309, Average Loss: 0.5652,MSE:0.49237255536196756
Epoch 310, Average Loss: 0.5654,MSE:0.4984518022755161
Epoch 311, Average Loss: 0.5640,MSE:0.4887421263065189
Epoch 312, Average Loss: 0.5640,MSE:0.4974765215570107
Epoch 313, Average Loss: 0.5652,MSE:0.4985065401038155
Epoch 314, Average Loss: 0.5613,MSE:0.48516979700028895
Epoch 315, Average Loss: 0.5627,MSE:0.48887550071608277
Epoch 316, Average Loss: 0.5603,MSE:0.49014481675196436
Epoch 317, Average Loss: 0.5599,MSE:0.49262747214678676
Epoch 318, Average Loss: 0.5608,MSE:0.4798320465106517
Ep

Epoch 449, Average Loss: 0.5252,MSE:0.44574770868839697
Epoch 450, Average Loss: 0.5226,MSE:0.4374747435931116
Epoch 451, Average Loss: 0.5180,MSE:0.43396288481298834
Epoch 452, Average Loss: 0.5181,MSE:0.4492287138755433
Epoch 453, Average Loss: 0.5335,MSE:0.4472656191494316
Epoch 454, Average Loss: 0.5295,MSE:0.43613609321331603
Epoch 455, Average Loss: 0.5250,MSE:0.42817548456704246
Epoch 456, Average Loss: 0.5210,MSE:0.4401831533331424
Epoch 457, Average Loss: 0.5207,MSE:0.43065154531318695
Epoch 458, Average Loss: 0.5160,MSE:0.4375519964477047
Epoch 459, Average Loss: 0.5132,MSE:0.42444718906600026
Epoch 460, Average Loss: 0.5157,MSE:0.4481226414234843
Epoch 461, Average Loss: 0.5184,MSE:0.43680107375988736
Epoch 462, Average Loss: 0.5153,MSE:0.42778881237953903
Epoch 463, Average Loss: 0.5186,MSE:0.4410879035125487
Epoch 464, Average Loss: 0.5158,MSE:0.42644169247280805
Epoch 465, Average Loss: 0.5131,MSE:0.43101925196647645
Epoch 466, Average Loss: 0.5155,MSE:0.42716942497245036

In [16]:
# 保存模型
PATH = './rating_regressor_100.pth'
torch.save(model.state_dict(), PATH)

In [17]:
for i,mse in enumerate(Mse):
    print(f'Epoch{i+1},MSE:{mse}')

Epoch1,MSE:1.1314000712901353
Epoch2,MSE:0.9893959114834666
Epoch3,MSE:0.9135288128580898
Epoch4,MSE:0.9480244465880096
Epoch5,MSE:0.9040041882034391
Epoch6,MSE:0.9248678304813802
Epoch7,MSE:0.8657799772314727
Epoch8,MSE:0.8704155466578901
Epoch9,MSE:0.8364718932341785
Epoch10,MSE:0.8426222413949669
Epoch11,MSE:0.8694272248327732
Epoch12,MSE:0.8402617111448198
Epoch13,MSE:0.8119284650109708
Epoch14,MSE:0.8600026292067021
Epoch15,MSE:0.8142544409230351
Epoch16,MSE:0.8075948779158294
Epoch17,MSE:0.8250312206327916
Epoch18,MSE:0.8033192804407329
Epoch19,MSE:0.8301366760080681
Epoch20,MSE:0.8119040590371936
Epoch21,MSE:0.7827515818532557
Epoch22,MSE:0.789468156767264
Epoch23,MSE:0.7786395530339331
Epoch24,MSE:0.7753198533548042
Epoch25,MSE:0.7863074863614514
Epoch26,MSE:0.7637563189782203
Epoch27,MSE:0.7673834804572165
Epoch28,MSE:0.7678374211218209
Epoch29,MSE:0.7696959763046354
Epoch30,MSE:0.7710029992304742
Epoch31,MSE:0.7579973260758445
Epoch32,MSE:0.7489544375460595
Epoch33,MSE:0.7374